In [1]:
%load_ext autoreload
%autoreload 2
from indices import MainDataset
from neighbors import Neighborhoods
import functions
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from catboost import CatBoostRegressor
from catboost import Pool
from metric import metrics_stat

In [2]:
train_dataset = MainDataset("dataset/train.csv")
# test_dataset = MainDataset("dataset/test.csv", is_train = False)

/Users/ivan.bragin/projects/raif/venv/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3263: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [3]:
neighborhoods = Neighborhoods(train_dataset.index)
neighborhoods1 = Neighborhoods(train_dataset.index1)

In [ ]:
# for i, o in enumerate(train_dataset.all_objects[:10000]):
#     if i % 1000 == 0:
#         print(i)
#     neighbor = neighborhoods.get_haversine_closest(o, 7)
#     for f_name, f in functions.items():
#         prices = [f(o, x[0], x[1]) for x in neighbor]
#         o.neighbors_scores[f_name] = np.mean(prices)

In [137]:
for prefix, neighborhoods_item in [("", neighborhoods), ("P1_", neighborhoods1)]:
    for objects in [train_dataset.all_train_objects, train_dataset.all_val_objects]:
        for i, o in enumerate(objects):
    #         o.neighbors_scores = {}
            if o.row["price_type"] != 1:
                continue
            if i % 1000 == 0:
                print(i)
            neighbor = neighborhoods_item.get_haversine_closest(o, 10)
            for f_name, f in functions.functions.items():
                prices = [f(o, x[0], x[1]) for x in neighbor]
                prices = [x for x in prices if x > 0]
                if len(prices) > 0:
                    price_median = np.median(prices)
                    price_max = np.max(prices)
                    price_min = np.min(prices)

                else:
                    price_median = 0
                    price_max = 0
                    price_min = 0
                o.neighbors_scores[prefix + f_name+"_median"] = price_median
                o.neighbors_scores[prefix + f_name+"_max"] = price_max
                o.neighbors_scores[prefix + f_name+"_min"] = price_min


184000
185000
186000
69000
70000
184000
185000
186000
69000
70000


In [161]:
raw_dense_features = [
#     'osm_historic_points_in_0.005',
#     'osm_culture_points_in_0.0075',
        'osm_amenity_points_in_0.001',
       'osm_amenity_points_in_0.005', 'osm_amenity_points_in_0.0075',
       'osm_amenity_points_in_0.01', 'osm_building_points_in_0.001',
       'osm_building_points_in_0.005', 'osm_building_points_in_0.0075',
       'osm_building_points_in_0.01', 'osm_catering_points_in_0.001',
       'osm_catering_points_in_0.005', 'osm_catering_points_in_0.0075',
       'osm_catering_points_in_0.01', 'osm_city_closest_dist',
        'osm_city_nearest_population',
       'osm_crossing_closest_dist', 'osm_crossing_points_in_0.001',
       'osm_crossing_points_in_0.005', 'osm_crossing_points_in_0.0075',
       'osm_crossing_points_in_0.01', 'osm_culture_points_in_0.001',
       'osm_culture_points_in_0.005', 
       'osm_culture_points_in_0.01', 'osm_finance_points_in_0.001',
       'osm_finance_points_in_0.005', 'osm_finance_points_in_0.0075',
       'osm_finance_points_in_0.01', 'osm_healthcare_points_in_0.005',
       'osm_healthcare_points_in_0.0075', 'osm_healthcare_points_in_0.01',
        'osm_historic_points_in_0.0075',
       'osm_historic_points_in_0.01', 'osm_hotels_points_in_0.005',
       'osm_hotels_points_in_0.0075', 'osm_hotels_points_in_0.01',
       'osm_leisure_points_in_0.005', 'osm_leisure_points_in_0.0075',
       'osm_leisure_points_in_0.01', 'osm_offices_points_in_0.001',
       'osm_offices_points_in_0.005', 'osm_offices_points_in_0.0075',
       'osm_offices_points_in_0.01', 'osm_shops_points_in_0.001',
       'osm_shops_points_in_0.005', 'osm_shops_points_in_0.0075',
       'osm_shops_points_in_0.01', 'osm_subway_closest_dist',
       'osm_train_stop_closest_dist', 'osm_train_stop_points_in_0.005',
       'osm_train_stop_points_in_0.0075', 'osm_train_stop_points_in_0.01',
       'osm_transport_stop_closest_dist', 'osm_transport_stop_points_in_0.005',
       'osm_transport_stop_points_in_0.0075',
       'osm_transport_stop_points_in_0.01',
                      'reform_count_of_houses_1000', 'reform_count_of_houses_500',
       'reform_house_population_1000', 'reform_house_population_500',
       'reform_mean_floor_count_1000', 'reform_mean_floor_count_500',
       'reform_mean_year_building_1000', 'reform_mean_year_building_500',
       'total_square', "lat", "lng"
       ]

raw_cat_features = ['city', 'osm_city_nearest_name', 'region', 'realty_type', 'street', 'floor']

bad_f = set(["F_square_False_min", "F_square_False_median"])
def getX(o):
    x = []
    for f_name, score in o.neighbors_scores.items():
        if f_name not in bad_f and "P1_k" not in f_name:
            x.append(score)
    for n in raw_dense_features:
        x.append(o.row[n])
    for n in raw_cat_features:
        x.append(o.row[n])
    return pd.Series(x)


In [162]:
columns = []
for p in ["",
          "P1_"
         ]:
    for t in ["_median", "_min", "_max"]:
        columns.extend([p + f_name + t for f_name, f in functions.functions.items() ])
columns = [c for c in columns if c not in bad_f]
    # columns = []
columns.extend(raw_dense_features)
columns.extend(raw_cat_features)

In [163]:
# X0 = [getX(o) for o in train_dataset.all_objects[:1] if o.row["price_type"] == 0]
# X0 = pd.concat(X0, axis=1).T
# X0.columns=columns
# Y0 = [o.getY() for o in train_dataset.all_objects if o.row["price_type"] == 0]
# X0["floor"] = X0["floor"].astype("str")

In [164]:
X1_train = [getX(o) for o in train_dataset.all_train_objects if o.row["price_type"] == 1]
X1_train = pd.concat(X1_train, axis=1).T
X1_train.columns=columns
Y1_train = [o.getY() for o in train_dataset.all_train_objects if o.row["price_type"] == 1]
X1_train = X1_train.fillna(-999)

X1_val = [getX(o) for o in train_dataset.all_val_objects if o.row["price_type"] == 1]
X1_val = pd.concat(X1_val, axis=1).T
X1_val.columns=columns
Y1_val = [o.getY() for o in train_dataset.all_val_objects if o.row["price_type"] == 1]
X1_val = X1_val.fillna(-999)

In [165]:
# Xtest1 = [getX(o) for o in test_dataset.all_objects]
# Xtest1 = pd.concat(Xtest1, axis=1).T
# Xtest1.columns=columns
# Xtest1 = Xtest1.fillna(-999)

In [166]:

train_data = Pool(data=X1_train,
                  cat_features=raw_cat_features,
                  label=Y1_train)

val_data = Pool(data=X1_val,
                cat_features=raw_cat_features,
                  label=Y1_val)

In [167]:
cat_model = CatBoostRegressor(
    learning_rate=0.03,
    iterations=7000
)
cat_model.fit(train_data,eval_set=val_data, use_best_model=False)



0:	learn: 87593.9656314	test: 91308.1982954	best: 91308.1982954 (0)	total: 9.43ms	remaining: 1m 5s
1:	learn: 86831.2252896	test: 90759.3694254	best: 90759.3694254 (1)	total: 21.9ms	remaining: 1m 16s
2:	learn: 86093.1930420	test: 90166.2141178	best: 90166.2141178 (2)	total: 35.2ms	remaining: 1m 22s
3:	learn: 85345.4327852	test: 89590.7883044	best: 89590.7883044 (3)	total: 44.9ms	remaining: 1m 18s
4:	learn: 84585.7223799	test: 88967.3971745	best: 88967.3971745 (4)	total: 54.5ms	remaining: 1m 16s
5:	learn: 83869.0324196	test: 88410.7374487	best: 88410.7374487 (5)	total: 63.8ms	remaining: 1m 14s
6:	learn: 83178.6368683	test: 87819.2183660	best: 87819.2183660 (6)	total: 72.2ms	remaining: 1m 12s
7:	learn: 82554.3864893	test: 87324.3300700	best: 87324.3300700 (7)	total: 77.8ms	remaining: 1m 8s
8:	learn: 81920.5215513	test: 86776.4505122	best: 86776.4505122 (8)	total: 85.6ms	remaining: 1m 6s
9:	learn: 81287.7553455	test: 86184.5241544	best: 86184.5241544 (9)	total: 93.8ms	remaining: 1m 5s
10:	

84:	learn: 62600.5039449	test: 74563.4695599	best: 74563.4695599 (84)	total: 807ms	remaining: 1m 5s
85:	learn: 62360.6739648	test: 74553.9170954	best: 74553.9170954 (85)	total: 819ms	remaining: 1m 5s
86:	learn: 62270.2159618	test: 74531.0026157	best: 74531.0026157 (86)	total: 828ms	remaining: 1m 5s
87:	learn: 62144.3356603	test: 74494.6730829	best: 74494.6730829 (87)	total: 837ms	remaining: 1m 5s
88:	learn: 61900.9044137	test: 74451.3819221	best: 74451.3819221 (88)	total: 846ms	remaining: 1m 5s
89:	learn: 61824.7803401	test: 74405.4331692	best: 74405.4331692 (89)	total: 854ms	remaining: 1m 5s
90:	learn: 61725.6542454	test: 74375.3484555	best: 74375.3484555 (90)	total: 863ms	remaining: 1m 5s
91:	learn: 61609.3305719	test: 74334.0642940	best: 74334.0642940 (91)	total: 872ms	remaining: 1m 5s
92:	learn: 61519.5985772	test: 74304.9873006	best: 74304.9873006 (92)	total: 885ms	remaining: 1m 5s
93:	learn: 61428.8799449	test: 74261.7236612	best: 74261.7236612 (93)	total: 895ms	remaining: 1m 5s


170:	learn: 52903.5872393	test: 73146.3888767	best: 73146.3888767 (170)	total: 1.62s	remaining: 1m 4s
171:	learn: 52838.4784882	test: 73146.0285031	best: 73146.0285031 (171)	total: 1.63s	remaining: 1m 4s
172:	learn: 52835.6337241	test: 73145.2137923	best: 73145.2137923 (172)	total: 1.64s	remaining: 1m 4s
173:	learn: 52397.8880439	test: 73176.0671526	best: 73145.2137923 (172)	total: 1.65s	remaining: 1m 4s
174:	learn: 52087.4078172	test: 73234.6791164	best: 73145.2137923 (172)	total: 1.66s	remaining: 1m 4s
175:	learn: 52011.5931992	test: 73238.2925953	best: 73145.2137923 (172)	total: 1.67s	remaining: 1m 4s
176:	learn: 51924.8663544	test: 73218.1320629	best: 73145.2137923 (172)	total: 1.68s	remaining: 1m 4s
177:	learn: 51789.1878572	test: 73207.5704758	best: 73145.2137923 (172)	total: 1.69s	remaining: 1m 4s
178:	learn: 51633.6912204	test: 73175.3285712	best: 73145.2137923 (172)	total: 1.7s	remaining: 1m 4s
179:	learn: 51596.5112030	test: 73164.8855926	best: 73145.2137923 (172)	total: 1.71

258:	learn: 45580.1082419	test: 72865.5852653	best: 72747.3093333 (223)	total: 2.42s	remaining: 1m 2s
259:	learn: 45468.3624728	test: 72861.2125400	best: 72747.3093333 (223)	total: 2.43s	remaining: 1m 3s
260:	learn: 45358.2080354	test: 72874.4569238	best: 72747.3093333 (223)	total: 2.45s	remaining: 1m 3s
261:	learn: 45337.9582242	test: 72875.2261214	best: 72747.3093333 (223)	total: 2.46s	remaining: 1m 3s
262:	learn: 45322.6551172	test: 72851.3616463	best: 72747.3093333 (223)	total: 2.47s	remaining: 1m 3s
263:	learn: 45282.9324762	test: 72849.6361978	best: 72747.3093333 (223)	total: 2.48s	remaining: 1m 3s
264:	learn: 45241.7145018	test: 72856.2262204	best: 72747.3093333 (223)	total: 2.49s	remaining: 1m 3s
265:	learn: 45170.3947477	test: 72823.8683832	best: 72747.3093333 (223)	total: 2.5s	remaining: 1m 3s
266:	learn: 45132.4407161	test: 72788.2026886	best: 72747.3093333 (223)	total: 2.51s	remaining: 1m 3s
267:	learn: 45074.2323120	test: 72774.8755314	best: 72747.3093333 (223)	total: 2.52

347:	learn: 39511.7722164	test: 72981.3635919	best: 72686.5733198 (276)	total: 3.43s	remaining: 1m 5s
348:	learn: 39428.2475931	test: 72994.3557731	best: 72686.5733198 (276)	total: 3.44s	remaining: 1m 5s
349:	learn: 39398.0528835	test: 72992.7846427	best: 72686.5733198 (276)	total: 3.45s	remaining: 1m 5s
350:	learn: 39326.5037566	test: 73036.1045974	best: 72686.5733198 (276)	total: 3.46s	remaining: 1m 5s
351:	learn: 39319.8213654	test: 73033.3327178	best: 72686.5733198 (276)	total: 3.47s	remaining: 1m 5s
352:	learn: 39216.5341238	test: 73026.4922890	best: 72686.5733198 (276)	total: 3.48s	remaining: 1m 5s
353:	learn: 39116.6893959	test: 73022.6580995	best: 72686.5733198 (276)	total: 3.49s	remaining: 1m 5s
354:	learn: 39097.0146204	test: 73020.9348547	best: 72686.5733198 (276)	total: 3.5s	remaining: 1m 5s
355:	learn: 39080.9904369	test: 73022.6979747	best: 72686.5733198 (276)	total: 3.52s	remaining: 1m 5s
356:	learn: 39005.5475593	test: 73016.0835411	best: 72686.5733198 (276)	total: 3.53

...

6594:	learn: 3256.2894018	test: 76108.5017957	best: 72686.5733198 (276)	total: 1m 14s	remaining: 4.59s
6595:	learn: 3255.5547752	test: 76108.0532754	best: 72686.5733198 (276)	total: 1m 14s	remaining: 4.58s
6596:	learn: 3255.3247180	test: 76108.0965429	best: 72686.5733198 (276)	total: 1m 14s	remaining: 4.57s
6597:	learn: 3254.9278430	test: 76108.2021923	best: 72686.5733198 (276)	total: 1m 14s	remaining: 4.56s
6598:	learn: 3254.2934768	test: 76108.2114107	best: 72686.5733198 (276)	total: 1m 14s	remaining: 4.54s
6599:	learn: 3252.7568505	test: 76108.0932345	best: 72686.5733198 (276)	total: 1m 14s	remaining: 4.53s
6600:	learn: 3252.3759934	test: 76107.9236723	best: 72686.5733198 (276)	total: 1m 14s	remaining: 4.52s
6601:	learn: 3252.2373314	test: 76107.8237064	best: 72686.5733198 (276)	total: 1m 14s	remaining: 4.51s
6602:	learn: 3252.0998981	test: 76107.8395154	best: 72686.5733198 (276)	total: 1m 14s	remaining: 4.5s
6603:	learn: 3251.7696913	test: 76107.7197127	best: 72686.5733198 (276)	to

6683:	learn: 3205.4501857	test: 76110.3014853	best: 72686.5733198 (276)	total: 1m 15s	remaining: 3.58s
6684:	learn: 3204.9965443	test: 76110.1500826	best: 72686.5733198 (276)	total: 1m 15s	remaining: 3.57s
6685:	learn: 3204.8587762	test: 76110.0956247	best: 72686.5733198 (276)	total: 1m 15s	remaining: 3.56s
6686:	learn: 3204.4694910	test: 76110.0597948	best: 72686.5733198 (276)	total: 1m 15s	remaining: 3.55s
6687:	learn: 3204.0743357	test: 76110.1030586	best: 72686.5733198 (276)	total: 1m 15s	remaining: 3.54s
6688:	learn: 3203.8542549	test: 76110.1869969	best: 72686.5733198 (276)	total: 1m 15s	remaining: 3.52s
6689:	learn: 3203.3772378	test: 76109.8504124	best: 72686.5733198 (276)	total: 1m 15s	remaining: 3.51s
6690:	learn: 3203.1106923	test: 76109.7672780	best: 72686.5733198 (276)	total: 1m 15s	remaining: 3.5s
6691:	learn: 3202.4849057	test: 76109.7848491	best: 72686.5733198 (276)	total: 1m 15s	remaining: 3.49s
6692:	learn: 3200.1829140	test: 76109.7542118	best: 72686.5733198 (276)	to

6780:	learn: 3152.1018835	test: 76113.6114534	best: 72686.5733198 (276)	total: 1m 16s	remaining: 2.48s
6781:	learn: 3151.1491697	test: 76113.3638846	best: 72686.5733198 (276)	total: 1m 16s	remaining: 2.47s
6782:	learn: 3150.5508333	test: 76113.2842414	best: 72686.5733198 (276)	total: 1m 16s	remaining: 2.46s
6783:	learn: 3150.0063512	test: 76113.5847308	best: 72686.5733198 (276)	total: 1m 16s	remaining: 2.44s
6784:	learn: 3149.8251122	test: 76113.5674914	best: 72686.5733198 (276)	total: 1m 16s	remaining: 2.43s
6785:	learn: 3149.5045798	test: 76113.7882825	best: 72686.5733198 (276)	total: 1m 16s	remaining: 2.42s
6786:	learn: 3148.8711026	test: 76114.1982419	best: 72686.5733198 (276)	total: 1m 16s	remaining: 2.41s
6787:	learn: 3148.2339484	test: 76114.3456018	best: 72686.5733198 (276)	total: 1m 16s	remaining: 2.4s
6788:	learn: 3147.7875923	test: 76114.4563542	best: 72686.5733198 (276)	total: 1m 16s	remaining: 2.39s
6789:	learn: 3147.7453612	test: 76114.4041434	best: 72686.5733198 (276)	to

6875:	learn: 3095.1216547	test: 76115.2943012	best: 72686.5733198 (276)	total: 1m 17s	remaining: 1.4s
6876:	learn: 3094.5404658	test: 76115.6375938	best: 72686.5733198 (276)	total: 1m 17s	remaining: 1.39s
6877:	learn: 3093.8688869	test: 76115.8504899	best: 72686.5733198 (276)	total: 1m 17s	remaining: 1.38s
6878:	learn: 3092.9894304	test: 76115.9811359	best: 72686.5733198 (276)	total: 1m 17s	remaining: 1.37s
6879:	learn: 3092.6820397	test: 76116.0337228	best: 72686.5733198 (276)	total: 1m 17s	remaining: 1.36s
6880:	learn: 3092.1285442	test: 76116.7483263	best: 72686.5733198 (276)	total: 1m 17s	remaining: 1.34s
6881:	learn: 3091.1042867	test: 76116.8771648	best: 72686.5733198 (276)	total: 1m 17s	remaining: 1.33s
6882:	learn: 3090.6911646	test: 76117.0385870	best: 72686.5733198 (276)	total: 1m 17s	remaining: 1.32s
6883:	learn: 3090.3828094	test: 76116.9715958	best: 72686.5733198 (276)	total: 1m 17s	remaining: 1.31s
6884:	learn: 3089.9272336	test: 76116.9954223	best: 72686.5733198 (276)	to

6963:	learn: 3044.3340292	test: 76121.1867228	best: 72686.5733198 (276)	total: 1m 18s	remaining: 407ms
6964:	learn: 3043.8757284	test: 76120.9801777	best: 72686.5733198 (276)	total: 1m 18s	remaining: 396ms
6965:	learn: 3043.2792115	test: 76120.7788006	best: 72686.5733198 (276)	total: 1m 18s	remaining: 385ms
6966:	learn: 3042.4729763	test: 76120.9292952	best: 72686.5733198 (276)	total: 1m 18s	remaining: 373ms
6967:	learn: 3041.9540230	test: 76120.9860403	best: 72686.5733198 (276)	total: 1m 18s	remaining: 362ms
6968:	learn: 3041.6092443	test: 76120.9151143	best: 72686.5733198 (276)	total: 1m 18s	remaining: 351ms
6969:	learn: 3041.1163998	test: 76120.9920720	best: 72686.5733198 (276)	total: 1m 18s	remaining: 339ms
6970:	learn: 3040.9530211	test: 76120.9362482	best: 72686.5733198 (276)	total: 1m 18s	remaining: 328ms
6971:	learn: 3039.5328225	test: 76121.8623389	best: 72686.5733198 (276)	total: 1m 18s	remaining: 317ms
6972:	learn: 3038.7905934	test: 76121.4738009	best: 72686.5733198 (276)	t

In [168]:
pd.set_option("display.max_rows", None) 
cat_model.get_feature_importance(data=val_data,prettified=True)


,Feature Id,Importances
0,total_square,16.269508
1,osm_crossing_points_in_0.0075,6.578948
2,city,3.476919
3,region,2.773908
4,lng,2.692265
5,realty_type,2.196657
6,F_max_distance_2_median,2.111750
7,F_square_True_max,2.110679
8,F_total_square_9_median,2.085085
9,F_total_price_max,2.061659


In [171]:
price0_preds = cat_model.predict(X1_train )
metrics_stat(np.array(price0_preds) , np.array(Y1_train))

{'mape': 0.06487997506897154,
 'mdape': 0.046659088396693874,
 'rmse': 7332.831620217116,
 'r2': 0.992605456006106,
 'raif_metric': 0.03942296964332058}

In [187]:
price0_preds = cat_model.predict(X1_val, ntree_end = 7000)
metrics_stat( np.array(Y1_val), np.array(price0_preds)+1000)

{'mape': 0.346641844160379,
 'mdape': 0.20283977260856376,
 'rmse': 76106.41599731905,
 'r2': 0.31572434768281754,
 'raif_metric': 1.9005993730352104}

In [204]:
metrics_stat(np.array(Y1_val), np.array(price0_preds)-3000 )

{'mape': 0.32212857141392115,
 'mdape': 0.1863336255500876,
 'rmse': 76240.89689433275,
 'r2': 0.3133039655302404,
 'raif_metric': 1.6194434041303034}

In [ ]:
{'mape': 0.2846841731316204,
 'mdape': 0.18254635409880016,
 'rmse': 76901.36707587936,
 'r2': -0.4397250121252976,
 'raif_metric': 1.3497125985711709}

In [ ]:
{'mape': 0.3938137450820849,
 'mdape': 0.27994944633239843,
 'rmse': 93738.73635776664,
 'r2': -3.7693751005637317,
 'raif_metric': 2.492385311141158}

{'mape': 0.28628533681632,
 'mdape': 0.21764848060697264,
 'rmse': 84390.73853846012,
 'r2': -1.6484418274393575,
 'raif_metric': 1.552534528099827}

In [ ]:
{'mape': 0.4006764913433361,
 'mdape': 0.28324961990050085,
 'rmse': 94343.39152646794,
 'r2': -4.286807955207906,
 'raif_metric': 2.6756861412533905}

{'mape': 0.2994072204832106,
 'mdape': 0.23136175216191418,
 'rmse': 85205.80509381538,
 'r2': -1.798582387807691,
 'raif_metric': 1.7313695017675064}

In [ ]:
{'mape': 0.45565440900238763,
 'mdape': 0.30249343647803606,
 'rmse': 100782.64525168539,
 'r2': -9.279126502012321,
 'raif_metric': 3.080882843995905}

In [72]:
X1.iloc[:,126:]

,realty_type,street,floor
0,110,S5845,-999
1,10,S27968,-999
2,10,S23597,-999
3,10,S10587,-999
4,10,S11220,-999
5,10,S6752,2.0
6,10,S6703,-999
7,10,S27593,-999
8,10,S19643,-999
9,10,S28820,-999


In [106]:
{'mape': 0.22629749264385002,
 'mdape': 0.19447444760686572,
 'rmse': 37925.78682309723,
 'r2': 0.6907862804815164,
 'raif_metric': 1.2450550091553048}
Xtest1 = Xtest1.fillna(-999)
test_data = Pool(data=Xtest1,
                  cat_features=raw_cat_features)
price0_preds = cat_model.predict(test_data)+1000

In [107]:
price0_preds

array([48461.5722999 , 42790.83136049, 38694.06343706, ...,
       60992.25486903, 50207.84320211, 55239.20223526])

In [108]:
pd.DataFrame({"id": [o.row['id'] for o in test_dataset.all_objects],
             "per_square_meter_price": price0_preds
             }).to_csv("no_overfit.csv", index=False)